# Implementing CNN for classification using the MNIST dataset

**Imports**

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import keras

In [0]:
#getting train/test data and labels
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data() 

# # reshaping
if keras.backend.image_data_format() == "channels_first":
    train_images = train_images.reshape(train_images.shape[0], 1, 28, 28).astype("float32")
    test_images = test_images.reshape(test_images.shape[0], 1, 28, 28).astype("float32")
    input_shape = (1, 28, 28)
else:
    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")
    test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype("float32")
    input_shape = (28, 28, 1)

train_images /= 255.
test_images /= 255.



In [0]:
#One-hot encoding labels
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

In [0]:
validation_images = train_images[54000:]
train_images = train_images[0:54000]

validation_labels = train_labels[54000:]
train_labels = train_labels[0:54000]

**Image Augmentation to cover feature types that may not be covered in training data**

In [0]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [0]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      )

validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [0]:
train_generator = training_datagen.flow(train_images, train_labels, batch_size=32)
validation_generator = validation_datagen.flow(validation_images, validation_labels, batch_size=32 )

**Building the Model**

In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

In [11]:
from tensorflow.keras.optimizers import RMSprop
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0

In [0]:
# model.fit(train_generator, epochs=10, validation_data = validation_generator)